In [41]:
# imports
import pandas as pd
import numpy as np
import spotipy #pip install spotipy first
import os
from client import *
from spotipy.oauth2 import SpotifyClientCredentials # required to access user data
import spotipy.util as util
from datetime import datetime
from pathlib import Path

### Spotify Authorization

In [42]:
def get_token(scope=None): #get token and set scope later if want
    redirect_uri = "http://localhost:4000"
    token = util.prompt_for_user_token(user, scope, cid, secret, redirect_uri)
    return token

In [43]:
user = userID
secret = secret
cid = clientID
AutoRepeatAll_id = AutoRepeatAll_id

In [44]:
token = get_token(scope='playlist-modify-private') # can make public later
sp = spotipy.Spotify(auth=token)

### Initialize the Playlist RUN ONCE

In [45]:
# run once, use playlist_id to ADD the songs to
def initialize_playlist():
    playlist = sp.user_playlist_create(user, 'AutoRepeatAll', public = False)
    playlist_id = playlist["id"]
    return playlist_id

In [46]:
AutoRepeatAll_id = initialize_playlist()

### Playlist URLs and Make main_df

In [47]:
# playlist urls to get data from
playlist_urls =['37i9dQZF1EpoJIsGQLtjPv', '37i9dQZF1EppjlneYJWXxZ', '37i9dQZF1Epf11aYgU5Q4u', '37i9dQZF1EpoxPdVWcv3CP', '37i9dQZF1EpwZ0YRZwMO7Y', '37i9dQZF1EpoCGPknzdJwy', '37i9dQZF1EphQVRnk9uGI6', '37i9dQZF1Epg86odGM5hPi', '37i9dQZF1EpL8wYDpLK6E6', '37i9dQZF1EpfkWyPBlEQKr', '37i9dQZF1EpgvNPBi84lNH',
'37i9dQZF1Eps5kuCDsqVgL']
creator = "spotify"

In [48]:
# make df to hold items to be added to playlist?
columns = ["artist", "album", "track_name", "track_id"]
main_df = pd.DataFrame(columns = columns)

### Get the songs from the playlists

In [49]:
# gets playlist tracks from single playlist and makes a df with track data for single playlist

def playlist_to_df(creator, playlist_id):
    playlist_df = pd.DataFrame(columns=columns)
    playlist_items = sp.user_playlist_tracks(creator, playlist_id)["items"] # returns item detail for each song
    for track in playlist_items:
        track_features = {} # make dict for each track with appropriate info
        track_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        track_features["album"] = track["track"]["album"]["name"]
        track_features["track_name"] = track["track"]["name"]
        track_features["track_id"] = track["track"]["id"]
        
        track_df = pd.DataFrame(track_features, index=[0]) # make df from dict for each item
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True) # add it to the ma
        
    return playlist_df
#https://www.linkedin.com/pulse/extracting-your-fav-playlist-info-spotifys-api-samantha-jones/

In [50]:
# makes list of dfs for each playlist_id in playlist_urls list
# returns list of dfs

def make_playlist_df_list(playlist_urls):
    playlist_df_list = []
    for playlist in playlist_urls:
        playlist_df_list.append(playlist_to_df(creator, playlist))
    return playlist_df_list

In [51]:
df_playlist_list = make_playlist_df_list(playlist_urls)

In [52]:
# concats all track info into one main_df
# returns main_df, duplicates removed
# saves this df as backup

def add_playlist_dfs_to_df(df, playlist_list):
    df = pd.concat(playlist_list, axis=0, ignore_index=True)
    df = df.drop_duplicates()
    df.to_csv("./data/{}.csv".format(datetime.now().strftime("%Y_%m_%d")), index=False)
    return df

In [53]:
og_df = add_playlist_dfs_to_df(main_df, df_playlist_list)

### Make list of track_uris and split into batches

In [54]:
all_tracks = og_df["track_id"].tolist()

In [55]:
# split into batches:
# use yield for a generator function

def make_batches(l, n):
    for i in range(0, len(l), n): 
        yield l[i:i + n]

In [56]:
batches = list(make_batches(all_tracks, 99))

### Add tracks from main_df to AutoRepeatAll playlist

In [57]:
# in batches, add the songs to the playlist

def add_tracks_to_main_playlist():
    for batch in batches:
        sp.playlist_add_items(playlist_id=AutoRepeatAll_id, items=batch)
    return print("confirm tracks added in spotify app")

In [58]:
add_tracks_to_main_playlist()

confirm tracks added in spotify app


## Once the Original Containing Playlist is made...

### get orignal Playlist 

In [25]:
def read_latest_csv():
    fld = "./data"
    files = Path(fld).glob('*csv')
    latest = max(files, key=lambda f: f.stat().st_mtime)
    return pd.read_csv("{}".format(latest))

In [36]:
# get laatest csv track list:
most_recent_df = read_latest_csv()
most_recent_tracks = most_recent_df["track_id"].tolist()

In [37]:
most_recent_df

,artist,album,track_name,track_id
0,The Smiths,Rank,"Still Ill - Live in London, 1986",1EU12yfayMBmqQy3Q92kn1
1,Kanye West,The Life Of Pablo,FML,30Zcjs7pgEkmJA1lEbGSWT
2,Babyshambles,Down In Albion,Fuck Forever,41Z2joQXOwZRBB8V6zk0A8
3,Ian Pooley,Since Then. 2017,900 Degrees,3nBF7oaQBAY6M2eVuUCnYn
4,Sweet Trip,Seen/Unseen,KKMJ,15VdjtCwcVZmVZHRy63Zgj
...,...,...,...,...
350,The Smiths,Louder Than Bombs,Hand in Glove - 2011 Remaster,17OgrEnuBJlrm0eksr3hfd
351,Hozier,Hozier,To Be Alone,0fEKxq1mghyT0b867l4Jaf
352,Cloudy June,FU In My Head,FU In My Head,5ZSV1Y3Cq2HkGOYOwxpClP
353,Billie Eilish,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",my strange addiction,3Tc57t9l2O8FwQZtQOvPXK


most_recent_df

In [27]:
# make df to hold items to be added to playlist?
new_columns = ["artist", "album", "track_name", "track_id"]
new_df = pd.DataFrame(columns = new_columns)

In [28]:
new_playlist_list = make_playlist_df_list(playlist_urls)

In [29]:
new_df = add_playlist_dfs_to_df(new_df, new_playlist_list)

In [30]:
new_tracks = new_df['track_id'].tolist()

In [39]:
len(new_tracks)

355

### Add Only New Songs if Playlists Update

In [32]:
def add_new_tracks_only():
    unique_tracks = []
    for i in new_tracks:
        if i not in most_recent_tracks:
            unique_tracks.append(i)
        else: pass
    sp.playlist_add_items(playlist_id=AutoRepeatAll_id, items= unique_tracks)

In [33]:
add_new_tracks_only()

HTTP Error for POST to https://api.spotify.com/v1/playlists/2BmjcSAnnohokmXpCJBMdN/tracks with Params: {'position': None} returned 400 due to Error parsing JSON.


SpotifyException: http status: 400, code:-1 - https://api.spotify.com/v1/playlists/2BmjcSAnnohokmXpCJBMdN/tracks:
 Error parsing JSON., reason: None